In [13]:

using ClusterMeanField
using PyCall
using LinearAlgebra
using Printf
using InCoreIntegrals
using QCBase
using ActiveSpaceSolvers
using RDM

In [21]:
molecule = "
He 0.00 0.00 0.00
He 1.5 0.00 0.00
He 1.5 1.5 0.00
He 0.00 1.5 0.00
He 0.70 0.70 2.20
He 2.20 0.70 2.20
He 2.20 2.20 2.20
He 0.70 2.20 2.20
"
atoms = []
for (li,line) in enumerate(split(rstrip(lstrip(molecule)), "\n"))
    l = split(line)
    push!(atoms, Atom(li, l[1], parse.(Float64,l[2:4])))
end

#basis = "aug-cc-pvdz" #9 orbs on each He
# basis = "cc-pvdz" #5 orbs on each He
#basis = "cc-pvtz" # 6 orbs on each He
#basis = "sto-3g"
basis="6-31g"
# Create FermiCG.Molecule type
mol = Molecule(0,1,atoms,basis)

pyscf = pyimport("pyscf")
pymol = pyscf.gto.Mole(atom=molecule, spin=0, charge=0, basis=basis)
pymol.build()
mf = pyscf.scf.RHF(pymol).run()
s = mf.get_ovlp(pymol)
ClusterMeanField.pyscf_write_molden(mol, s, filename="ao_631g.molden")
lo = pyimport("pyscf.lo.orth")
lo_ao = lo.lowdin(s)
# println("size of Lowdin ortho AO's:", size(lo_ao))

# ClusterMeanField.pyscf_write_molden(mol, lo_ao, filename="lowdin_ao_631g.molden")

#write fci dump file from the modified mo coefficients
tools = pyimport("pyscf.tools")
tools.fcidump.from_mo(pymol, "fcidump.he08_ras", s)

converged SCF energy = -22.6864444244694
size of Lowdin ortho AO's:(16, 16)


In [19]:
#Can just read in pyscf dump file for integrals (once you have already run an scf calculation)
pyscf = pyimport("pyscf");
fcidump = pyimport("pyscf.tools.fcidump");
ctx = fcidump.read("/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/fcidump.he08_ras");
h = ctx["H1"];
g = ctx["H2"];
ecore = ctx["ECORE"];
g = pyscf.ao2mo.restore("1", g, size(h,2))

ints = InCoreInts(ecore,h,g);

In [11]:
#Define clusters and intial Fock space for inital CMF calc for 2 orbs each He
clusters_in = [(1:8),(9:16)]
init_fspace = [(4,4),(4,4)]
rdm1 = zeros(size(ints.h1))
na=8
nb=8

8

In [6]:
#Define clusters now using ClusterMeanField code
clusters = [MOCluster(i,collect(clusters_in[i])) for i = 1:length(clusters_in)]
display(clusters)

print(size(ints.h1))
rdm1 = zeros(size(ints.h1))

#do a CMF calculation to optimize cluster orbitals

@time e_cmf, U, d1 = ClusterMeanField.cmf_oo(ints, clusters, init_fspace, RDM1(n_orb(ints)),
    verbose=0, gconv=1e-8,tol_d1=1e-12,tol_ci=1e-14,sequential=true, max_iter_oo=100)

IDX001:DIM65536:001|002|003|004|005|006|007|008|
IDX002:DIM65536:009|010|011|012|013|014|015|016|
(16, 16)

Overwritten attributes  conv_tol_residual  of <class 'pyscf.fci.direct_spin1.FCISolver'>


 ooCMF Iter:    1 Total= -22.876153970278 Active= -45.594693272134 G=     1.48e-01


 ooCMF Iter:    2 Total= -22.877403445953 Active= -45.595942747809 G=     1.10e-02


 ooCMF Iter:    3 Total= -22.877409928849 Active= -45.595949230706 G=     1.48e-03


...

 ooCMF Iter:   51 Total= -22.877410245708 Active= -45.595949547564 G=     1.36e-07


 * Status: success

 * Candidate solution
    Final objective value:     -2.287741e+01

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 1.78e-14 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 7.76e-16 ≰ 0.0e+00
    |g(x)|                 = 3.95e-08 ≰ 1.0e-08

 * Work counters
    Seconds run:   136  (vs limit Inf)
    Iterations:    51
    f(x) calls:    129
    ∇f(x) calls:   129


 ooCMF Iter:   52 Total= -22.877410245708 Active= -45.595949547564 G=     1.36e-07
*ooCMF -22.87741025 


250.386630 seconds (261.38 M allocations: 17.178 GiB, 2.22% gc time, 44.99% compilation time)


ErrorException: getindex not defined for RDM1{Float64}

In [7]:
ansatze = [RASCIAnsatz(8, 4,4, (2,4,2), max_h=4, max_p=4), RASCIAnsatz(8,4,4,(2,4,2), max_h=4, max_p=4)] #FCI type RASCI calculation
display(ansatze)

rdm1 = zeros(size(ints.h1))
e_cmf, U_cmf, d1  = ClusterMeanField.cmf_oo_diis(ints, clusters, init_fspace, ansatze, RDM1(rdm1, rdm1), maxiter_oo = 400, tol_oo=1e-8,tol_d1=1e-9, tol_ci=1e-10, verbose=0, diis_start=1);
ints_cmf = orbital_rotation(ints,U_cmf)
C_cmf = Cact*U_cmf
e_cmf, U_n, d1_n = ClusterMeanField.cmf_oo_newton2(ints, clusters, init_fspace, ansatze,RDM1(rdm1, rdm1), maxiter_oo = 2500,
                           tol_oo=1e-8, 
                           tol_d1=1e-9, 
                           tol_ci=1e-10,
                           verbose=4, 
                           zero_intra_rots = true,
                           sequential=true)
ints_cmf = orbital_rotation(ints,U_n)
C_cmf = Cact*U_n


2-element Vector{RASCIAnsatz}:
 RASCIAnsatz(8, 4, 4, 70, 70, 4900, 4900, [2, 4, 2], 4, 4)
 RASCIAnsatz(8, 4, 4, 70, 70, 4900, 4900, [2, 4, 2], 4, 4)

 Solve OO-CMF with DIIS


MethodError: MethodError: no method matching invariant_orbital_rotations(::RASCIAnsatz)
Closest candidates are:
  invariant_orbital_rotations(!Matched::Vector) at ~/.julia/packages/ActiveSpaceSolvers/dpMhJ/src/Helpers.jl:60

In [ ]:
ansatze = [RASCIAnsatz(8, 4,4, (2,4,2), max_h=0, max_p=0), RASCIAnsatz(8,4,4,(2,4,2), max_h=0, max_p=0)]#CASCI type RASCI calculation
display(ansatze)
rdm1 = zeros(size(ints.h1))
e_cmf, U_cmf, d1  = ClusterMeanField.cmf_oo_diis(ints, clusters, init_fspace, ansatze, RDM1(rdm1, rdm1), maxiter_oo = 400, tol_oo=1e-8,tol_d1=1e-9, tol_ci=1e-10, verbose=0, diis_start=1);
ints_cmf = orbital_rotation(ints,U_cmf)
C_cmf = Cact*U_cmf
e_cmf, U_n, d1_n = ClusterMeanField.cmf_oo_newton2(ints, clusters, init_fspace, ansatze,RDM1(rdm1, rdm1), maxiter_oo = 2500,
                           tol_oo=1e-8, 
                           tol_d1=1e-9, 
                           tol_ci=1e-10,
                           verbose=4, 
                           zero_intra_rots = true,
                           sequential=true)
ints_cmf = orbital_rotation(ints,U_n)
C_cmf = Cact*U_n


In [ ]:
ansatze = [RASCIAnsatz(8, 4,4, (2,4,2), max_h=2, max_p=2), RASCIAnsatz(8,4,4,(2,4,2), max_h=2, max_p=2)] #double excitation RASCI calculation
display(ansatze)

rdm1 = zeros(size(ints.h1))
e_cmf, U_cmf, d1  = ClusterMeanField.cmf_oo_diis(ints, clusters, init_fspace, ansatze, RDM1(rdm1, rdm1), maxiter_oo = 400, tol_oo=1e-8,tol_d1=1e-9, tol_ci=1e-10, verbose=0, diis_start=1);
ints_cmf = orbital_rotation(ints,U_cmf)
C_cmf = Cact*U_cmf
e_cmf, U_n, d1_n = ClusterMeanField.cmf_oo_newton2(ints, clusters, init_fspace, ansatze,RDM1(rdm1, rdm1), maxiter_oo = 2500,
                           tol_oo=1e-8, 
                           tol_d1=1e-9, 
                           tol_ci=1e-10,
                           verbose=4, 
                           zero_intra_rots = true,
                           sequential=true)
ints_cmf = orbital_rotation(ints,U_n)
C_cmf = Cact*U_n





In [ ]:
ansatze = [RASCIAnsatz(8, 4,4, (2,4,2), max_h=1, max_p=1), RASCIAnsatz(8,4,4,(2,4,2), max_h=1, max_p=1)]#Single excitation RASCI calculation
display(ansatze)

rdm1 = zeros(size(ints.h1))
e_cmf, U_cmf, d1  = ClusterMeanField.cmf_oo_diis(ints, clusters, init_fspace, ansatze, RDM1(rdm1, rdm1), maxiter_oo = 400, tol_oo=1e-8,tol_d1=1e-9, tol_ci=1e-10, verbose=0, diis_start=1);
ints_cmf = orbital_rotation(ints,U_cmf)
C_cmf = Cact*U_cmf
e_cmf, U_n, d1_n = ClusterMeanField.cmf_oo_newton2(ints, clusters, init_fspace, ansatze,RDM1(rdm1, rdm1), maxiter_oo = 2500,
                           tol_oo=1e-8, 
                           tol_d1=1e-9, 
                           tol_ci=1e-10,
                           verbose=4, 
                           zero_intra_rots = true,
                           sequential=true)
ints_cmf = orbital_rotation(ints,U_n)
C_cmf = Cact*U_n

In [ ]:
ansatze = [RASCIAnsatz(8, 4,4, (2,4,2), max_h=3, max_p=3), RASCIAnsatz(8,4,4,(2,4,2), max_h=3, max_p=3)] 
display(ansatze)

rdm1 = zeros(size(ints.h1))
e_cmf, U_cmf, d1  = ClusterMeanField.cmf_oo_diis(ints, clusters, init_fspace, ansatze, RDM1(rdm1, rdm1), maxiter_oo = 400, tol_oo=1e-8,tol_d1=1e-9, tol_ci=1e-10, verbose=0, diis_start=1);
ints_cmf = orbital_rotation(ints,U_cmf)
C_cmf = Cact*U_cmf
e_cmf, U_n, d1_n = ClusterMeanField.cmf_oo_newton2(ints, clusters, init_fspace, ansatze,RDM1(rdm1, rdm1), maxiter_oo = 2500,
                           tol_oo=1e-8, 
                           tol_d1=1e-9, 
                           tol_ci=1e-10,
                           verbose=4, 
                           zero_intra_rots = true,
                           sequential=true)
ints_cmf = orbital_rotation(ints,U_n)
C_cmf = Cact*U_n